In [144]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [145]:
LABELS = {
    'emg_healthy.txt': 0,
    'emg_myopathy.txt': 1,
    'emg_neuropathy.txt': 2
}
label_names = ['healthy', 'myopathy', 'neuropathy']



In [146]:
def load_emg_column(file_path, column=1):
    signal = []
    with open(file_path, 'r', errors='ignore') as f:
        for line in f:
            try:
                parts = line.strip().split()
                signal.append(float(parts[column]))
            except:
                continue
    return np.array(signal)

In [147]:
def window_signal(signal, label, window_size=1000):
    X, y = [], []
    for i in range(0, len(signal) - window_size, window_size):
        win = signal[i:i+window_size]
        win = (win - np.mean(win)) / (np.std(win) + 1e-6)  # normalize
        X.append(win.reshape(-1, 1))
        y.append(label)
    return X, y

In [148]:
all_X, all_y = [], []
for fname, label in LABELS.items():
    sig = load_emg_column(fname)
    X, y = window_signal(sig, label)
    all_X.extend(X)
    all_y.extend(y)

X = np.array(all_X)
y = np.array(all_y)

print("X shape:", X.shape)  # (samples, 1000, 1)
print("y shape:", y.shape)

X shape: (307, 1000, 1)
y shape: (307,)


In [149]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


In [150]:
def build_model(input_shape=(1000, 1), num_classes=3):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv1D(16, 7, activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling1D(4)(x)
    x = tf.keras.layers.Conv1D(32, 5, activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(4)(x)
    x = tf.keras.layers.Conv1D(64, 3, activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [151]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(16)
val_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(16)

model = build_model()
model.fit(train_ds, validation_data=val_ds, epochs=20)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.4535 - loss: 1.0181 - val_accuracy: 0.8280 - val_loss: 0.9289
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7765 - loss: 0.9262 - val_accuracy: 0.8280 - val_loss: 0.8191
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8015 - loss: 0.8075 - val_accuracy: 0.8280 - val_loss: 0.6327
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8046 - loss: 0.6068 - val_accuracy: 0.8280 - val_loss: 0.4365
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.7971 - loss: 0.4298 - val_accuracy: 0.8280 - val_loss: 0.3132
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7798 - loss: 0.3754 - val_accuracy: 0.9032 - val_loss: 0.2471
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9050 - loss: 0.2526 - val_accuracy: 0.9677 - val_loss: 0.1872
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9687 - loss: 0.1532 - val_accuracy: 0.9785 - v

In [152]:
y_pred = np.argmax(model.predict(X_test), axis=1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_names))

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step

Classification Report:
              precision    recall  f1-score   support

     healthy       0.88      1.00      0.94        15
    myopathy       1.00      1.00      1.00        33
  neuropathy       1.00      0.96      0.98        45

    accuracy                           0.98        93
   macro avg       0.96      0.99      0.97        93
weighted avg       0.98      0.98      0.98        93



In [154]:
model.save("emg_classifier_txt.keras")